In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import create_react_agent


In [2]:
llm = ChatOpenAI(model="gpt-4o-mini")


In [ ]:
username = 'root'
password = 'mindstix'
host = 'localhost'  # or your database host
port = '3306'       # default MySQL port
database = 'edb'
table_name = 'amazon_sale_report'
sql_string = f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}'

# db = SQLDatabase.from_uri("sqlite:///Chinook.db")
db = SQLDatabase.from_uri(sql_string)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

In [9]:
# print(db.dialect)
# print(db.get_usable_table_names())
# print(db.run("SELECT * FROM amazon_sale_report LIMIT 10;"))
# print(db.get_table_info())
# print(db.get_context())

In [10]:
with open('db_schema.txt', 'r') as file:
    # Read the contents of the file and store it in a variable
    db_schema = file.read()

In [ ]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db, prompt=PROMPT)
response = chain.invoke({"input": "give info with id 403-9615377-8133951", "table_info": db_schema})
response

In [ ]:
db_chain.invoke(input={"query": "give info with id 403-9615377-8133951"})
# res = db_chain.invoke(input={"messages": [HumanMessage(content="give info with id 403-9615377-8133951")]})

In [ ]:
system = """
Using the provided user question and the database schema , generate a valid MySQL query. Ensure that the query directly addresses the user's request and can handle complex logic if necessary. The goal is to retrieve relevant information from the database while accounting for potential joins, subqueries, or conditions as required.

User question:
""".format(
    db_schema=db_schema
)
print(system)
system_message = SystemMessage(content=system)


agent = create_react_agent(llm, tools, messages_modifier=system_message)

In [ ]:

_mysql_prompt = f"""You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Below is the name and description of each table tables that you can use:

['amazon_sale_report', 'cloud_warehouse_compersion', 'customer_sales', 'p_l_march_2021', 'p_l_may_2022', 'stock']


Question: 

"""


system_message = SystemMessage(content=_mysql_prompt)


In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.chains import create_sql_query_chain
from langchain.chains.sql_database.query import create_sql_query_chain
execute_query = QuerySQLDataBaseTool(db=db)
# write_query = create_sql_query_chain(llm, db, prompt=_DEFAULT_TEMPLATE)
chain = db_chain | execute_query
db_chain.invoke({"query": "give info with id 403-9615377-8133951"})

In [ ]:
chain.invoke({"query": "give info with id 403-9615377-8133951"})

In [7]:
res = agent.invoke(input={"messages": [HumanMessage(content="Describe the given tables")]})

In [ ]:
for s in agent.stream(
    {"messages": [HumanMessage(content="top 5 category with highest sales")]}
):
    print(s)
    print("----")

In [ ]:
for s in agent.stream(
    {"messages": [HumanMessage(content="calculate the coorelation between size of product and its has selled succesfully ")]}
):
    print(s)
    print("----")

In [ ]:
for s in agent.stream(
    {"messages": [HumanMessage(content="the sales are down in june 2022 as compared to may 2022, what may be the possible reasons behind this. can you provide few datapoints and numeric results of analysis based on the data available to back your reasons")]}
):
    print(s)
    print("----")

In [ ]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


class Table(BaseModel):
    """Table in SQL database."""

    name: str = Field(description="Name of table in SQL database.")


table_names = "\n".join(db.get_usable_table_names())
system = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
The tables are:

{table_names}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{input}"),
    ]
)
llm_with_tools = llm.bind_tools([Table])
output_parser = PydanticToolsParser(tools=[Table])

table_chain = prompt | llm_with_tools | output_parser

table_chain.invoke({"input": "What are all the genres of Alanis Morisette songs"})

In [ ]:
# [QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10fa9ee20>),
#  InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10fa9ee20>),
#  ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10fa9ee20>),
#  QuerySQLCheckerTool(description='Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10fa9ee20>, llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10fa8e160>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10fa9e310>, root_client=<openai.OpenAI object at 0x1075a2310>, root_async_client=<openai.AsyncOpenAI object at 0x10fa8e1c0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['dialect', 'query'], input_types={}, partial_variables={}, template='\n{query}\nDouble check the {dialect} query above for common mistakes, including:\n- Using NOT IN with NULL values\n- Using UNION when UNION ALL should have been used\n- Using BETWEEN for exclusive ranges\n- Data type mismatch in predicates\n- Properly quoting identifiers\n- Using the correct number of arguments for functions\n- Casting to the correct data type\n- Using the proper columns for joins\n\nIf there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.\n\nOutput the final SQL query only.\n\nSQL Query: '), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10fa8e160>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10fa9e310>, root_client=<openai.OpenAI object at 0x1075a2310>, root_async_client=<openai.AsyncOpenAI object at 0x10fa8e1c0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}))]

In [14]:
import pandas as pd

# Load CSV into DataFrame
df = pd.read_csv('P  L March 2021 Cleaned - P  L March 2021.csv')

# Replace blank values with the word "NULL"
df.replace('', 'NULL', inplace=True)

# Save DataFrame back to CSV
df.to_csv('output2222.csv', index=False)


In [ ]:
import pandas as pd

# Define the input file path and the desired chunk size
input_csv = '/Users/apple/Documents/hackathon_ai/ai-commerce-insights/Orders - Sheet1.csv'  # Replace with your CSV file path
chunk_size = 500000  # Number of lines per chunk

# Read the CSV file in chunks and process each chunk
for i, chunk in enumerate(pd.read_csv(input_csv, chunksize=chunk_size, sep=',', na_filter=False)):
    # Replace blank values with the word "NULL"
    chunk.replace('', 'NULL', inplace=True)
    
    # Save each processed chunk to a separate CSV file
    output_csv = f"chunk_{i+1}.csv"  # Naming the output files
    chunk.to_csv(output_csv, index=False)
    print(f"Chunk {i+1} saved as {output_csv}")


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Define database connection parameters
username = 'root'
password = 'mindstix'
host = 'localhost'  # or your database host
port = '3306'       # default MySQL port
database = 'edb'
table_name = 'amazon_sale_report'
csv_file_path = 'hii.csv'

# Create a SQLAlchemy engine for the MySQL database
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}')

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Write the DataFrame to a MySQL table
df.to_sql(table_name, con=engine, if_exists='append', index=False)

print(f"Data from {csv_file_path} has been successfully imported to the {table_name} table in {database} database.")


In [5]:
def clean_query(query):
    prefix = "SQLQuery:"
    if query.startswith(prefix):
        return query[len(prefix):].strip()
    return query

# Example usage
# query = 'SQLQuery: SELECT COUNT("EmployeeId") AS "EmployeeCount" FROM "Employee"'
# cleaned_query = clean_query(query)
# print(cleaned_query)


In [8]:
# pip install -U langchain langchain-community langchain-openai
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})

In [6]:
response = clean_query(response)

In [ ]:
db.run(response)

In [ ]:
# from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
from langchain_experimental.sql import SQLDatabaseChain


db.get_usable_table_names()
db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=False)
db_chain.run("give nuber of order with amount less than 1000")

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_sql_query_chain


In [ ]:
system = """You are a {dialect} expert. Given an input question, create a syntactically correct {dialect} query to run.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per {dialect}. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Only use the following tables:
{table_info}

Write an initial draft of the query. Then double check the {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

Use format:

First draft: <<FIRST_DRAFT_QUERY>>
Final answer: <<FINAL_ANSWER_QUERY>>
"""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}")]
).partial(dialect=db.dialect)


def parse_final_answer(output: str) -> str:
    return output.split("Final answer: ")[1]


chain = create_sql_query_chain(llm, db, prompt=prompt)
prompt.pretty_print()

In [ ]:
db.dialect

In [ ]:
query = chain.invoke(
    {
        "question": "give info with id 403-9615377-8133951"
    }
)
print(query)

In [ ]:
db.run(query)

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

In [ ]:
from langchain.chains import create_sql_query_chain

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
response

In [32]:
def clean_query(query):
    prefix = "SQLQuery:"
    if query.startswith(prefix):
        return query[len(prefix):].strip()
    return query

response = clean_query(response)

In [ ]:
db.run(response)

In [ ]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="top 5 category with highest sales")]}
):
    print(s)
    print("----")

In [ ]:
username = 'root'
password = 'mindstix'
host = 'localhost'  # or your database host
port = '3306'       # default MySQL port
database = 'edb'
table_name = 'amazon_sale_report'
sql_string = f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}'
from langchain_community.utilities import SQLDatabase

# db = SQLDatabase.from_uri("sqlite:///Chinook.db")
db = SQLDatabase.from_uri(sql_string)


In [ ]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine, text

# Database connection details
username = 'root'
password = 'mindstix'
host = 'localhost'  # or your database host
port = '3306'       # default MySQL port
database = 'edb'
table_name = 'customer_sales'

# SQLAlchemy connection string
sql_string = f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}'

# Create a SQLAlchemy engine
engine = create_engine(sql_string)

# Connect to the database and execute the DROP TABLE command
with engine.connect() as connection:
    drop_table_query = f"DROP TABLE IF EXISTS {table_name};"
    connection.execute(text(drop_table_query))

print(f"Table {table_name} deleted successfully (if it existed).")


In [10]:
print("SELECT \nCOUNT(CASE WHEN `Status` LIKE 'Shipped%' THEN 1 END) / COUNT(*) * 100 AS `FulfillmentRate`\nFROM `amazon_sale_report`\nWHERE `OrderDate` BETWEEN '2022-06-01' AND '2022-06-30';")

SELECT 
COUNT(CASE WHEN `Status` LIKE 'Shipped%' THEN 1 END) / COUNT(*) * 100 AS `FulfillmentRate`
FROM `amazon_sale_report`
WHERE `OrderDate` BETWEEN '2022-06-01' AND '2022-06-30';
